<a href="https://colab.research.google.com/github/dbright123/Dbot-Advance/blob/main/lstm_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd





In [7]:
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

trade_active = mt5.initialize()

print(trade_active)

if not trade_active:
    print('Initialization failed, check internet connection. You must have Meta Trader 5 installed.')
    mt5.shutdown()

else:
    print(mt5.account_info()._asdict())
    print("\n")
    print(mt5.terminal_info()._asdict())
    print("\n")
    print(mt5.symbols_total())

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4200
True
{'login': 213644473, 'trade_mode': 0, 'leverage': 1000, 'limit_orders': 500, 'margin_so_mode': 0, 'trade_allowed': True, 'trade_expert': True, 'margin_mode': 2, 'currency_digits': 2, 'fifo_close': False, 'balance': 200.0, 'credit': 0.0, 'profit': 0.0, 'equity': 200.0, 'margin': 0.0, 'margin_free': 200.0, 'margin_level': 0.0, 'margin_so_call': 25.0, 'margin_so_so': 15.0, 'margin_initial': 0.0, 'margin_maintenance': 0.0, 'assets': 0.0, 'liabilities': 0.0, 'commission_blocked': 0.0, 'name': 'micheal bright omage ', 'server': 'OctaFX-Demo', 'currency': 'USD', 'company': 'Octa Markets Incorporated'}


{'community_account': False, 'community_connection': False, 'connected': True, 'dlls_allowed': False, 'trade_allowed': True, 'tradeapi_disabled': False, 'email_enabled': False, 'ftp_enabled': False, 'notifications_enabled': False, 'mqid': True, 'build': 5370, 'maxbars': 100000000, 'codepage': 0, 'ping_last

In [3]:
account = mt5.account_info()
terminal = mt5.terminal_info()

print(account.equity)

if(terminal.connected == True or terminal.trade_allowed == True):
    print("AI is successfully functional")
else:
    print("Please make sure metatrade 5 has internet and algo Trade is Turn On")

200.0
AI is successfully functional


In [8]:
t_s = "BTCUSD"
market = mt5.copy_rates_from_pos(t_s, mt5.TIMEFRAME_M1, 0, 10000000)
#display(market)
print(market.shape)

print(t_s)




(3577655,)
BTCUSD


In [5]:
market

array([(1311242400, 1.3690000e+01, 1.3780000e+01, 1.3400000e+01, 1.3610000e+01,  69,    0, 0),
       (1311328800, 1.3610000e+01, 1.3810000e+01, 1.3450000e+01, 1.3700000e+01,  64,    0, 0),
       (1311415200, 1.3700000e+01, 1.3760000e+01, 1.3510000e+01, 1.3680000e+01,  49,    0, 0),
       ...,
       (1761498900, 1.1353208e+05, 1.1357874e+05, 1.1352328e+05, 1.1356182e+05, 148, 2253, 0),
       (1761498960, 1.1355832e+05, 1.1358547e+05, 1.1351382e+05, 1.1353182e+05, 139, 2300, 0),
       (1761499020, 1.1353222e+05, 1.1353262e+05, 1.1346487e+05, 1.1349600e+05, 150, 2268, 0)],
      dtype=[('time', '<i8'), ('open', '<f8'), ('high', '<f8'), ('low', '<f8'), ('close', '<f8'), ('tick_volume', '<u8'), ('spread', '<i4'), ('real_volume', '<u8')])

In [9]:
data = []
for i in range(len(market)):
    data.append([market[i][1],market[i][2],market[i][3],market[i][4]])
df = pd.DataFrame(data, columns=["open", "high","low", "close"])
df.to_csv("Generated"+t_s+" dbot.csv", index=False)

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
print(tf.__version__)


2.10.1


In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import os
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scaler3d2d import preprocess_and_save_scalers,transform_data, inverse_transform_data,create_sequences



In [3]:
from tensorflow.keras.optimizers import SGD,Adam
import matplotlib.pyplot as plt



from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization, LayerNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model
import keras

In [4]:
t_symbol = ["XAUUSD"]

In [5]:


def engineer_features_5min_btc(df):
    """
    Input: df with columns ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    Output: df with 68 features
    """

    # GROUP 1: Core Price Features
    df['typical_price'] = (df['high'] + df['low'] + df['close']) / 3

    # GROUP 7: Support/Resistance Features
    df['pivot_point'] = (df['high'].shift(1) + df['low'].shift(1) + df['close'].shift(1)) / 3
    
    return df

# Usage:
# df = pd.read_csv('btcusd_5min.csv')
# df = engineer_features_5min_btc(df)

In [6]:
n = 0
m_label = "Generated"+t_symbol[n]
train_df = pd.read_csv(m_label+ " dbot.csv")
train_df = engineer_features_5min_btc(train_df[["open","high","low","close"]])[100:]

In [7]:
train_df

,open,high,low,close,typical_price,pivot_point
100,1277.77,1282.78,1274.08,1280.99,1279.283333,1280.190000
101,1279.83,1280.79,1274.33,1276.51,1277.210000,1279.283333
102,1275.15,1290.80,1275.15,1280.86,1282.270000,1277.210000
103,1281.08,1287.28,1280.04,1282.39,1283.236667,1282.270000
104,1282.46,1296.38,1282.09,1289.21,1289.226667,1283.236667
...,...,...,...,...,...,...
663219,4102.07,4106.72,4101.78,4106.72,4105.073333,4102.036667
663220,4106.53,4109.63,4105.80,4108.38,4107.936667,4105.073333
663221,4108.29,4111.53,4106.87,4111.32,4109.906667,4107.936667
663222,4111.40,4113.50,4111.40,4112.62,4112.506667,4109.906667


In [8]:

#train_df_test = train_dfs_test[n]
print(m_label)
SEQ_LEN = 120 # length of input sequence (timesteps). Typical choices: 30, 60, 90
n_rows, n_features = train_df.values.shape


X, y = create_sequences(train_df.values/33333, SEQ_LEN, 3, 3)  # X: (n_samples, SEQ_LEN, n_features), y: (n_samples, n_features)
train_df = None
n_samples = X.shape[0]
#print(X[-1])
print(y[-1])

GeneratedXAUUSD
[0.3700225 0.3701395 0.3700234]


In [9]:
print(X[-1])

[[0.36524795 0.36544775 0.36508415 0.36528575 0.36527255 0.36534425]
 [0.36526955 0.36529655 0.36501755 0.36516155 0.36515855 0.36527255]
 [0.36517325 0.36524795 0.36481955 0.36491765 0.36499505 0.36515855]
 [0.36491855 0.36512285 0.36392764 0.36418594 0.36441214 0.36499505]
 [0.36417964 0.36547205 0.36408244 0.36541625 0.36499025 0.36441214]
 [0.36541535 0.36556746 0.36508595 0.36530015 0.36531785 0.36499025]
 [0.36529385 0.36550626 0.36513275 0.36527585 0.36530495 0.36531785]
 [0.36527315 0.36531275 0.36496265 0.36502295 0.36509945 0.36530495]
 [0.36502385 0.36528755 0.36492665 0.36510485 0.36510635 0.36509945]
 [0.36509675 0.36541985 0.36494645 0.36540635 0.36525755 0.36510635]
 [0.36540545 0.36541265 0.36489695 0.36501845 0.36510935 0.36525755]
 [0.36504275 0.36527045 0.36494825 0.36526055 0.36515975 0.36510935]
 [0.36526235 0.36591036 0.36520475 0.36589416 0.36566976 0.36515975]
 [0.36588876 0.36590676 0.36536135 0.36538025 0.36554946 0.36566976]
 [0.36540545 0.36556206 0.36529385

In [10]:
#X, y = preprocess_and_save_scalers(X, y)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0, shuffle=False)
X_val, X_test, y_val,  y_test = train_test_split(X_test, y_test, test_size=0.01, random_state=0, shuffle=False)
X = None
y = None
seq_len = X_train.shape[1]
n_features = X_train.shape[2]
output_dim = y_train.shape[1]

i = Input(shape=(seq_len, n_features))

X = Bidirectional(LSTM(64, return_sequences=True))(i)
X = BatchNormalization()(X)
X = Dropout(0.3)(X)

X = Bidirectional(LSTM(64))(X)
X = BatchNormalization()(X)
X = Dropout(0.3)(X)

X = Dense(50, activation="relu")(X)
X = Dropout(0.3)(X)

X = Dense(output_dim)(X)

model = Model(i, X)
model.summary()


I0000 00:00:1761406870.412129   27051 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 120, 6)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 120, 200)       │        85,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 120, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 120, 200)       │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 120, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 120, 200)       │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 120, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 200)            │       240,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           453 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 841,003 (3.21 MB)

 Trainable params: 839,803 (3.20 MB)

 Non-trainable params: 1,200 (4.69 KB)

In [12]:

es = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=2)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=6, min_lr=1e-6, verbose=2)
mc = ModelCheckpoint(m_label + 'lstm_best.keras', monitor='val_loss', save_best_only=True, verbose=2)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae',tf.keras.metrics.RootMeanSquaredError(),'mape','msle'])
r = model.fit(
  X_train, y_train,
  validation_data=(X_val, y_val),
  epochs=100,
  batch_size=256,
  callbacks=[es, rlr, mc ],
  verbose=1

)

Epoch 1/100


2025-10-25 16:41:17.442991: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400


1736/1736 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.0082 - mae: 0.0295 - mape: 21.9639 - msle: 0.0023 - root_mean_squared_error: 0.0696
Epoch 1: val_loss improved from None to 0.00086, saving model to GeneratedXAUUSDlstm_best.keras
1736/1736 ━━━━━━━━━━━━━━━━━━━━ 249s 140ms/step - loss: 0.0014 - mae: 0.0119 - mape: 8.7072 - msle: 4.8541e-04 - root_mean_squared_error: 0.0370 - val_loss: 8.5870e-04 - val_mae: 0.0162 - val_mape: 6.0374 - val_msle: 5.0109e-04 - val_root_mean_squared_error: 0.0293 - learning_rate: 0.0010
Epoch 2/100
1736/1736 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 3.9358e-05 - mae: 0.0047 - mape: 3.3692 - msle: 2.9844e-05 - root_mean_squared_error: 0.0063
Epoch 2: val_loss improved from 0.00086 to 0.00067, saving model to GeneratedXAUUSDlstm_best.keras
1736/1736 ━━━━━━━━━━━━━━━━━━━━ 245s 141ms/step - loss: 3.5968e-05 - mae: 0.0045 - mape: 3.2368 - msle: 2.7296e-05 - root_mean_squared_error: 0.0060 - val_loss: 6.6685e-04 - val_mae: 0.0156 - val_mape: 6.2927 - val_msle:

KeyboardInterrupt: 

In [ ]:
plt.title("Loss on data")
plt.plot(r.history['loss'], label="loss")
plt.plot(r.history['val_loss'], label="val_loss")
plt.legend()
plt.show()

plt.title("MAE on data")
plt.plot(r.history['mae'], label="mae")
plt.plot(r.history['val_mae'], label="val_mae")
plt.legend()
plt.show()

plt.title("mape on data")
plt.plot(r.history['mape'], label="mape")
plt.plot(r.history['val_mape'], label="val_mape")
plt.legend()
plt.show()

plt.title("msle on data")
plt.plot(r.history['msle'], label="msle")
plt.plot(r.history['val_msle'], label="val_msle")
plt.legend()
plt.show()

plt.title("root_mean_squared_error on data")
plt.plot(r.history['root_mean_squared_error'], label="root_mean_squared_error")
plt.plot(r.history['val_root_mean_squared_error'], label="val_root_mean_squared_error")
plt.legend()
plt.show()




: 

In [ ]:
m_label = "GeneratedXAUUSD"
best_model = load_model(m_label + 'lstm_best.keras')

: 

In [ ]:
y_pred = best_model.predict(X_test)

: 

In [ ]:
print(y_pred.shape)

: 

In [ ]:
#_,y_test = inverse_transform_data(scaled_y=y_test)
#_,y_pred = inverse_transform_data(scaled_y=y_pred)
print(y_test, " comparing to ", y_pred)

: 

In [ ]:
# 1. Calculate the error (the gap between actual and predicted)
error = y_test - y_pred

# 2. Calculate the average gap (the bias)
average_gap = np.mean(error)
print(f"Average Gap (Bias): {average_gap:.7f}")

# 3. Add the average gap to your predictions to create a corrected version
y_pred_corrected = y_pred + average_gap

# --- Verification ---
# Let's check the first predicted value vs. the first actual value
print("\n--- Example of Correction ---")
print(f"Original Prediction: {y_pred[0, 0]:.7f}")
print(f"Corrected Prediction: {y_pred_corrected[0, 0]:.7f}")
print(f"Actual Value:        {y_test[0, 0]:.7f}")

: 

In [ ]:
print("R^2 value for ", m_label)
print(r2_score(y_test, y_pred))

: 

In [ ]:




plt.figure(figsize=(12, 6))
plt.plot(y_test[-50:, 0], label='Actual Close Price', color='blue')
plt.plot(y_pred[-50:, 0], label='Predicted Close Price', color='red', linestyle='--')
plt.plot(y_pred_corrected[-50:, 0], label='corrected Predicted Close Price', color='gray', linestyle='--')
plt.title('Actual vs. Predicted Close Price')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.savefig('actual_vs_predicted.png')
plt.show()


: 